# Validation of the PyPSA-Africa network

This notebooks investigates the data quality of the African network by 
comparing raw Open Street Map, processed PyPSA and World Bank data.
Looking at Nigeria only, we further compare the data to official Transmission
System Operator data.

The following quantities will be reviewed:
- inputs used by the pypsa-model:
  - network length
  - network topography
  - substation amount

To properly reproduce the findings obtained in this notebook,
please run the full snakemake workflow for the Africa.
To do so, please set ``countries = ["Africa"]`` in the ``config.yaml`` file.

## Preparation

### Import packages

In [ ]:
# import packages

import logging
import os

import pypsa
import pandas as pd
import geopandas as gpd
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import OSM
import cartopy.feature as cfeature

# import geoplot
import pandas as pd

# import fiona  # ;help(fiona.open)
# import requests
from pandas.io.json import json_normalize  # convert json into dataframe

import shapely, shapely.prepared
from shapely.geometry import Point, LineString
from shapely.wkt import loads
from shapely.validation import make_valid
from pyproj import Geod  # to calculate length globally "Geodesic calculations"

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)

### Set main directory to root folder

In [ ]:
# set current folders
import sys

sys.path.append("../../")  # adds path to $ .../pypsa-africa
from scripts._helpers import sets_path_to_root

sets_path_to_root("pypsa-africa")  # moves path to root

## 1. Load data 




#### World Bank Data

Note: This dataset has been updated with transmission lines for the MENA region. This is the most complete and up-to-date open map of Africa's electricity grid network. This dataset serves as an updated and improved replacement for the Africa Infrastructure Country Diagnostic (AICD) data that was published in 2007. 

Coverage. This dataset includes planned and existing grid lines for all continental African countries and Madagascar, as well as the Middle East region. The lines range in voltage from sub-kV to 700 kV EHV lines, though there is a very large variation in the completeness of data by country. An interactive tool has been created for exploring this data, the Africa Electricity Grids Explorer. 

Sources. The primary sources for this dataset are as follows: 
- Africa Infrastructure Country Diagnostic (AICD) 
- OSM © OpenStreetMap contributors 
- For MENA: Arab Union of Electricity and country utilities. 
- For West Africa: West African Power Pool (WAPP) 
- GIS database 
- World Bank projects archive and 
- International Bank for Reconstruction and Development (IBRD) maps 
- There were many additional sources for specific countries and areas. This information is contained in the files of this dataset, and can also be found by browsing the individual country datasets, which contain more extensive information. 

Limitations. Some of the data, notably that from the AICD and from World Bank project archives, may be very out of date. Where possible this has been improved with data from other sources, but in many cases this wasn't possible. This varies significantly from country to country, depending on data availability. Thus, many new lines may exist which aren't shown, and planned lines may have completely changed or already been constructed. 
The data that comes from World Bank project archives has been digitized from PDF maps. This means that these lines should serve as an indication of extent and general location, but shouldn't be used for precisely location grid lines.

https://energydata.info/dataset/africa-electricity-transmission-and-distribution-grid-map-2017

In [ ]:
url = "https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/144823/africagrid20170906final.geojson"
df_lines_world_bank = gpd.read_file(url).to_crs(epsg=3857)
values = ["Jordan", "Syria", "Lebanon", "Iraq", "Saudi Arabia", "Oman", "Yemen"]
df_lines_world_bank = df_lines_world_bank[
    df_lines_world_bank.country.isin(values) == False
]
df_lines_world_bank_existing = df_lines_world_bank[
    df_lines_world_bank.status == "Existing"
]
df_lines_world_bank_planned = df_lines_world_bank[
    df_lines_world_bank.status == "Planned"
]
df_lines_world_bank_existing_nigeria = df_lines_world_bank_existing[
    df_lines_world_bank_existing.country == "Nigeria"
]

#### Raw Open Street Map Data
Created by our python script `download_osm_data.py`. Generate this data by running the pypsa-africa workflow through `snakemake -j8 download_osm_data` (see Snakefile if run fails)

In [ ]:
# substations
substations_OSMraw_path = os.getcwd() + "/data/raw/africa_all_raw_substations.geojson"
df_substations_raw_osm_africa = gpd.read_file(substations_OSMraw_path).to_crs(epsg=3857)
df_substations_raw_osm_nigeria = df_substations_raw_osm_africa[
    df_substations_raw_osm_africa.Country == "NG"
]

# lines
lines_OSMraw_path = os.getcwd() + "/data/raw/africa_all_raw_lines.geojson"
df_lines_raw_osm_africa = gpd.read_file(lines_OSMraw_path).to_crs(epsg=3857)
df_lines_raw_osm_nigeria = df_lines_raw_osm_africa[
    df_lines_raw_osm_africa.Country == "NG"
]

#### Cleaned Open Street Map Data 
Created by our python script `clean_osm_data.py`. Generate this data by running the pypsa-africa workflow through `snakemake -j8 clean_osm_data` (see Snakefile if run fails)

In [ ]:
# substations
substations_OSMclean_path = os.getcwd() + "/data/clean/africa_all_substations.geojson"
df_substations_clean_osm_africa = gpd.read_file(substations_OSMclean_path)
df_substations_clean_osm_africa["geometry"] = (
    df_substations_clean_osm_africa["geometry"].apply(make_valid).to_crs(epsg=3857)
)
df_substations_clean_osm_nigeria = df_substations_clean_osm_africa[
    df_substations_clean_osm_africa.country == "NG"
]

# lines
lines_OSMclean_path = os.getcwd() + "/data/clean/africa_all_lines.geojson"
df_lines_clean_osm_africa = gpd.read_file(lines_OSMclean_path)
df_lines_clean_osm_africa["geometry"] = (
    df_lines_clean_osm_africa["geometry"].apply(make_valid).to_crs(epsg=3857)
)
df_lines_clean_osm_nigeria = df_lines_clean_osm_africa[
    df_lines_clean_osm_africa.country == "NG"
]

#### Country shape

In [ ]:
countries_shape_path = os.getcwd() + "/resources/country_shapes.geojson"
africa_shape = gpd.read_file(countries_shape_path)
nigeria_shape = africa_shape.set_index("name").loc["NG"].geometry

## 2. Network length

The line length is given by voltage level in [km]. Note: when multiple circuits are available, the length of the line is multiplied accordingly. The World Bank data does not include directly circuit information, however, since maps sometimes show lines in parallel some circuits are included.

#### World bank

In [ ]:
### AFRICA
africa_length = df_lines_world_bank_existing
africa_length["voltage_kV"] = africa_length["voltage_kV"].astype(int)
africa_length["length_km"] = africa_length["length_km"].astype(int)
africa_length["length_km_epsg3857"] = africa_length.geometry.length / 1000

df = africa_length.groupby(by=["voltage_kV"]).sum()
lengths = pd.DataFrame()
lengths["110-220kV"] = df.loc[(df.index >= 110) & (df.index < 220)].sum()
lengths["220-380kV"] = df.loc[(df.index >= 220) & (df.index < 380)].sum()
lengths["380+kV"] = df.loc[(df.index >= 380)].sum()
lengths["scope"] = "Africa"
lengths["reference"] = "World Bank"

### NIGERIA
nigeria_length = df_lines_world_bank_existing_nigeria
nigeria_length["voltage_kV"] = nigeria_length["voltage_kV"].astype(int)
nigeria_length["length_km"] = nigeria_length["length_km"].astype(int)
nigeria_length["length_km_epsg3857"] = nigeria_length.geometry.length / 1000

df = nigeria_length.groupby(by=["voltage_kV"]).sum()
lengths_ng = pd.DataFrame()
lengths_ng["110-220kV"] = df.loc[(df.index >= 110) & (df.index < 220)].sum()
lengths_ng["220-380kV"] = df.loc[(df.index >= 220) & (df.index < 380)].sum()
lengths_ng["380+kV"] = df.loc[(df.index >= 380)].sum()
lengths_ng["scope"] = "Nigeria"
lengths_ng["reference"] = "World Bank"
lengths_ng

### COMBINE OUTPUT
world_bank_lengths = pd.concat([lengths, lengths_ng])
world_bank_lengths

#### Raw OSM

In [ ]:
df_lines_raw_osm_africa["length_by_geometry"] = df_lines_raw_osm_africa.to_crs(
    epsg=3857
).geometry.length

In [ ]:
# AFRICA
df_lines_raw_osm_africa["cables_fix"] = df_lines_raw_osm_africa["tags.cables"].copy()
df_lines_raw_osm_africa.loc[
    df_lines_raw_osm_africa["tags.cables"].isnull()
    | df_lines_raw_osm_africa["tags.cables"].isna(),
    "cables_fix",
] = 3  # when NaN or None, set default value
df_lines_raw_osm_africa["length_km"] = (
    df_lines_raw_osm_africa.to_crs(epsg=3857).geometry.length
    * df_lines_raw_osm_africa["cables_fix"].astype(float)
    / 3
)
length_raw_osm = (
    df_lines_raw_osm_africa.groupby(by=["tags.voltage"]).length_km.sum() / 1000
)
voltage = pd.to_numeric(
    pd.DataFrame(length_raw_osm).reset_index()["tags.voltage"], errors="coerce"
)
df_raw_osm = pd.DataFrame(length_raw_osm)
df_raw_osm.index = voltage

lengths_africa = pd.DataFrame()
df = df_raw_osm
lengths_africa["110-220kV"] = df.loc[(df.index >= 110000) & (df.index < 220000)].sum()
lengths_africa["220-380kV"] = df.loc[(df.index >= 220000) & (df.index < 380000)].sum()
lengths_africa["380+kV"] = df.loc[(df.index >= 380000)].sum()
lengths_africa["scope"] = "Africa"
lengths_africa["reference"] = "Open Street Map (raw)"

### NIGERIA
df_lines_raw_osm_nigeria["cables_fix"] = df_lines_raw_osm_nigeria["tags.cables"].copy()
df_lines_raw_osm_nigeria.loc[
    df_lines_raw_osm_nigeria["tags.cables"].isnull()
    | df_lines_raw_osm_nigeria["tags.cables"].isna(),
    "cables_fix",
] = 3  # when NaN or None, set default value
df_lines_raw_osm_nigeria["length_km"] = (
    df_lines_raw_osm_nigeria.to_crs(epsg=3857).geometry.length
    * df_lines_raw_osm_nigeria["cables_fix"].astype(float)
    / 3
)
length_raw_osm = (
    df_lines_raw_osm_nigeria.groupby(by=["tags.voltage"]).length_km.sum() / 1000
)
voltage = pd.to_numeric(
    pd.DataFrame(length_raw_osm).reset_index()["tags.voltage"], errors="coerce"
)
df_lines_raw_osm_nigeria = pd.DataFrame(length_raw_osm)
df_lines_raw_osm_nigeria.index = voltage

lengths_ng = pd.DataFrame()
df = df_lines_raw_osm_nigeria
lengths_ng["110-220kV"] = df.loc[(df.index >= 110000) & (df.index < 220000)].sum()
lengths_ng["220-380kV"] = df.loc[(df.index >= 220000) & (df.index < 380000)].sum()
lengths_ng["380+kV"] = df.loc[(df.index >= 380000)].sum()
lengths_ng["scope"] = "Nigeria"
lengths_ng["reference"] = "Open Street Map (raw)"

### COMBINE OUTPUT
osm_raw_lengths = pd.concat([lengths_africa, lengths_ng])
osm_raw_lengths

#### Clean OSM

In [ ]:
### AFRICA
df = df_lines_clean_osm_africa
df["length"] = df.to_crs(epsg=3857).geometry.length * df.circuits / 1000
voltage = df.groupby(by=["voltage"]).length.sum()
df_clean_osm = pd.DataFrame(voltage)
lengths_africa = pd.DataFrame()
df = df_clean_osm
lengths_africa["110-220kV"] = df.loc[(df.index >= 110000) & (df.index < 220000)].sum()
lengths_africa["220-380kV"] = df.loc[(df.index >= 220000) & (df.index < 380000)].sum()
lengths_africa["380+kV"] = df.loc[(df.index >= 380000)].sum()
lengths_africa["scope"] = "Africa"
lengths_africa["reference"] = "Open Street Map (clean)"

### NIGERIA
df = df_lines_clean_osm_nigeria
df["length"] = df.to_crs(epsg=3857).geometry.length * df.circuits / 1000
voltage = df.groupby(by=["voltage"]).length.sum()
df_clean_osm = pd.DataFrame(voltage)
lengths_ng = pd.DataFrame()
df = df_clean_osm
lengths_ng["110-220kV"] = df.loc[(df.index >= 110000) & (df.index < 220000)].sum()
lengths_ng["220-380kV"] = df.loc[(df.index >= 220000) & (df.index < 380000)].sum()
lengths_ng["380+kV"] = df.loc[(df.index >= 380000)].sum()
lengths_ng["scope"] = "Nigeria"
lengths_ng["reference"] = "Open Street Map (clean)"

### COMBINE OUTPUT
osm_clean_lengths = pd.concat([lengths_africa, lengths_ng])
osm_clean_lengths

## 3. Network topology

In this subsection, the data publicly on the network power system are compared. In particular, the main data that are being compared are:
- Layout of the network as shown by images (Section 1.1)
- Total length of the line (Section 1.2)

The data used for the comparison have been taken from public reliable sources, with strong focus on the website of the Nigerian Transmission operator: Nigerian Transmission Company ([TCN](https://www.tcn.org.ng/)).

In the Section 1.1, the network layout obtained by the workflow is drawn to be possibly compared with the image as shown in the public website of the [Nigerian Transmission System Operator](https://www.tcn.org.ng/). The image below, dated 2016, available from the [Nigerian Transmission System Operator](https://www.tcn.org.ng/), depicts the national power grid of Nigeria.

![Nigerian transmission system](../../images/Nigeria_data/Nigeria_network_map.png)
[Source link](https://www.tcn.org.ng/page_repository.php)

In [ ]:
import contextily as cx  # Need to be installed `pip3 install contextily`
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib as mpl

# adapt all sizes
size = 1
# Create the basemap that is called by ax
ax = africa_shape.to_crs(epsg="3857").plot(
    figsize=(20 * size, 20 * size), alpha=0.5, facecolor="white", edgecolor="whitesmoke"
)

# Create open street map background map
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, alpha=0.4)

# Create open street map background map
df = df_lines_clean_osm_africa[df_lines_clean_osm_africa.country == "NG"]
df = df_lines_clean_osm_africa[df_lines_clean_osm_africa.voltage >= 110000]

# Scale and format stuff
blues = cm.get_cmap("Blues")
dfact = 0.28  # impacts min-max differences of weight, transparency and color
widthscale = 10 * size
df["normalized_v"] = (1 - dfact) * (df["voltage"] - df["voltage"].min()) / (
    df["voltage"].max() - df["voltage"].min()
) + dfact
line = df
normalized_voltage = df.normalized_v

line.plot(
    ax=ax,
    alpha=normalized_voltage,
    linewidth=normalized_voltage * widthscale,
    color=blues(normalized_voltage + 0.1),
    legend=True,
)
ax.set_title(
    "Data: Open Street Map",
    fontsize=40 * size,
    y=1.0,
    pad=-40 * size,
    fontweight="bold",
    color="grey",
)
ax.set_axis_off()

# Colorbar
cmap = blues
norm = mpl.colors.Normalize(
    vmin=df["voltage"].min() / 1000, vmax=df["voltage"].max() / 1000
)
cbar = plt.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax,
    orientation="vertical",
    extend="max",
    shrink=0.9,
)
cbar.set_label("Voltage in kV", fontsize=16, color="dimgrey")
# ticks & edge modification of cbar
cbar.ax.yaxis.set_tick_params(color="grey")
plt.setp(plt.getp(cbar.ax.axes, "yticklabels"), color="dimgrey")
cbar.outline.set_edgecolor(None)  # set colorbar edgecolor

In [ ]:
from matplotlib import cm
import contextily as cx  # Need to be installed `pip3 install contextily`

# adapt all sizes
size = 1
# Create the basemap that is called by ax
ax = africa_shape.to_crs(epsg="3857").plot(
    figsize=(20 * size, 20 * size), alpha=0.5, facecolor="white", edgecolor="whitesmoke"
)

# Create open street map background map
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, alpha=0.4)

# Create open street map background map
df = df_lines_world_bank_existing[df_lines_world_bank_existing.voltage_kV >= 110]

# Scale and format stuff
reds = cm.get_cmap("Reds")
dfact = 0.28  # impacts min-max differences of weight, transparency and color
widthscale = 10 * size
df["normalized_v"] = (1 - dfact) * (df["voltage_kV"] - df["voltage_kV"].min()) / (
    df["voltage_kV"].max() - df["voltage_kV"].min()
) + dfact
line = df
normalized_voltage = df.normalized_v

line.plot(
    ax=ax,
    alpha=normalized_voltage,
    linewidth=normalized_voltage * widthscale,
    color=reds(normalized_voltage + 0.1),
)
ax.set_title(
    "Data: World Bank Group",
    fontsize=40 * size,
    y=1.0,
    pad=-40 * size,
    fontweight="bold",
    color="grey",
)
ax.set_axis_off()

# Colorbar
cmap = reds
norm = mpl.colors.Normalize(vmin=df["voltage_kV"].min(), vmax=df["voltage_kV"].max())
# ticks = np.linspace(df["voltage_kV"].min(), df["voltage_kV"].max(), 100, endpoint=True)

cbar = plt.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    ax=ax,
    orientation="vertical",
    extend="max",
    shrink=0.9,
)
cbar.set_label("Voltage in kV", fontsize=16, color="dimgrey")
# ticks & edge modification of cbar
cbar.ax.yaxis.set_tick_params(color="grey")
plt.setp(plt.getp(cbar.ax.axes, "yticklabels"), color="dimgrey")
cbar.outline.set_edgecolor(None)  # set colorbar edgecolor

In [ ]:
from matplotlib import cm
import contextily as cx  # Need to be installed `pip3 install contextily`
import matplotlib.pyplot as plt
import matplotlib as mpl

# adapt all sizes
size = 0.5
# Create the basemap that is called by ax
ax = (
    africa_shape[africa_shape.name == "NG"]
    .to_crs(epsg="3857")
    .plot(
        figsize=(20 * size, 20 * size),
        alpha=0.5,
        facecolor="white",
        edgecolor="whitesmoke",
    )
)

# Create open street map background map
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, alpha=0.4)

# Create open street map background map
df0 = df_lines_clean_osm_africa[df_lines_clean_osm_africa.country == "NG"]
df = df0[df0.voltage >= 110000]

# Scale and format stuff
blues = cm.get_cmap("Blues")
dfact = 0.6  # impacts min-max differences of weight, transparency and color
widthscale = 10 * size
df["normalized_v"] = (1 - dfact) * (df["voltage"] - df["voltage"].min()) / (
    df["voltage"].max() - df["voltage"].min()
) + dfact
line = df
normalized_voltage = df.normalized_v

line.plot(
    ax=ax,
    alpha=normalized_voltage,
    linewidth=normalized_voltage * widthscale,
    color=blues(normalized_voltage + 0.1),
)

# Legend
ax.set_title(
    "Data: Open Street Map",
    fontsize=40 * size,
    y=1.0,
    pad=-40 * size,
    fontweight="bold",
    color="grey",
)
ax.set_axis_off()
plt.plot(
    0.6, label="132kV", color=blues(normalized_voltage.unique()[0] + 0.1), linewidth=3
)
plt.plot(
    1, label="330kV", color=blues(normalized_voltage.unique()[1] + 0.1), linewidth=5
)
plt.rc("legend", fontsize=15)
ax.legend()

In [ ]:
from matplotlib import cm
import contextily as cx  # Need to be installed `pip3 install contextily`

# adapt all sizes
size = 0.5
# Create the basemap that is called by ax
ax = (
    africa_shape[africa_shape.name == "NG"]
    .to_crs(epsg="3857")
    .plot(
        figsize=(20 * size, 20 * size),
        alpha=0.5,
        facecolor="white",
        edgecolor="whitesmoke",
    )
)

# Create open street map background map
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron, alpha=0.4)

# Create open street map background map
df0 = df_lines_world_bank_existing[df_lines_world_bank_existing.country == "Nigeria"]
df = df0[df0.voltage_kV >= 110]

# Scale and format stuff
reds = cm.get_cmap("Reds")
dfact = 0.6  # impacts min-max differences of weight, transparency and color
widthscale = 10 * size
df["normalized_v"] = (1 - dfact) * (df["voltage_kV"] - df["voltage_kV"].min()) / (
    df["voltage_kV"].max() - df["voltage_kV"].min()
) + dfact
line = df
normalized_voltage = df.normalized_v

line.plot(
    ax=ax,
    alpha=normalized_voltage,
    linewidth=normalized_voltage * widthscale,
    color=reds(normalized_voltage + 0.1),
)

# Legend
ax.set_title(
    "Data: World Bank Group",
    fontsize=40 * size,
    y=1.0,
    pad=-40 * size,
    fontweight="bold",
    color="grey",
)
ax.set_axis_off()
plt.plot(
    0.6, label="132kV", color=reds(normalized_voltage.unique()[0] + 0.1), linewidth=3
)
plt.plot(
    1, label="330kV", color=reds(normalized_voltage.unique()[1] + 0.1), linewidth=5
)
plt.rc("legend", fontsize=15)
ax.legend()